In [1]:
import cv2
from math import sin, cos, radians
from PIL import Image
import os

In [2]:
dir_path = "C:/Users/chinm/Documents/Courses/Sem4/CV/Project/1/"

images_dir = "data/originalPics/"
fddb_file_path = "data/FDDB-folds/"
img_format = ".jpg"

fddb_fold_files = [i for i in os.listdir(dir_path+fddb_file_path) if 'ellipse' not in i]
fddb_fold_ellipse_files = [i for i in os.listdir(dir_path+fddb_file_path) if 'ellipse' in i]

output_dir = "data/output/"
bbox_fdd_folds = "bbox-FDDB-folds/"
bbox_faces_dir = "faces/"
bbox_non_faces_dir = "non_faces/"
face_dim = (20, 20)

In [3]:
def convert_ellipse_to_bbox():
    for ellipse_file in fddb_fold_ellipse_files:
        print('****************Parsing {}**********************'.format(ellipse_file))
        with open(dir_path+fddb_file_path+ellipse_file) as f:
            lines = [line.rstrip('\n') for line in f]
        line_num = 0
        with open(dir_path + output_dir + bbox_fdd_folds + ellipse_file.split('ellipse')[0] + "BoundingBox.txt", "w") as bounding_box_file:
            while line_num<len(lines):
                img_path = lines[line_num]

                with Image.open(dir_path + images_dir + img_path + img_format) as img:
                    img_width, img_height = img.size
                num_faces = int(lines[line_num+1])

                bounding_boxes = []
                for i in range(num_faces):
                    major_axis_radius, minor_axis_radius, angle, center_x, center_y, _ = [float(l) for l in lines[line_num+2+i].split()]                 
                    
                    rect_height = 2 * major_axis_radius * (cos(radians(abs(angle))))
                    rect_width = 2 * minor_axis_radius * (cos(radians(abs(angle))))

                    left_x = int(max(0, center_x - rect_width/2))
                    left_y = int(max(0, center_y - rect_height/2))
                    right_x = int(min(img_width-1, center_x + rect_width/2))
                    right_y = int(min(img_height-1, center_y + rect_height/2))
                    
                    bounding_boxes.append([left_x, left_y, right_x, right_y])

                bounding_box_file.write(img_path+"\n")
                bounding_box_file.write(str(num_faces)+"\n")
                for bb_box in bounding_boxes:
                    bounding_box_file.write(" ".join([str(i) for i in bb_box])+"\n")
                line_num += num_faces + 2
        print('****************Completed Parsing {}**********************'.format(ellipse_file))

In [4]:
convert_ellipse_to_bbox()

****************Parsing FDDB-fold-01-ellipseList.txt**********************
****************Completed Parsing FDDB-fold-01-ellipseList.txt**********************
****************Parsing FDDB-fold-02-ellipseList.txt**********************
****************Completed Parsing FDDB-fold-02-ellipseList.txt**********************
****************Parsing FDDB-fold-03-ellipseList.txt**********************
****************Completed Parsing FDDB-fold-03-ellipseList.txt**********************
****************Parsing FDDB-fold-04-ellipseList.txt**********************
****************Completed Parsing FDDB-fold-04-ellipseList.txt**********************
****************Parsing FDDB-fold-05-ellipseList.txt**********************
****************Completed Parsing FDDB-fold-05-ellipseList.txt**********************
****************Parsing FDDB-fold-06-ellipseList.txt**********************
****************Completed Parsing FDDB-fold-06-ellipseList.txt**********************
****************Parsing FDDB-fold-07-ell

In [5]:
bounding_box_files = [f for f in os.listdir(dir_path + output_dir + bbox_fdd_folds) if 'BoundingBox' in f]    
len(bounding_box_files)

10

In [8]:
#extract face data in the form of bbox
def extract_faces_from_images():
    for bounding_box_file in bounding_box_files:
        with open(dir_path + output_dir + bbox_fdd_folds + bounding_box_file) as f:
            lines = [l.strip("\n") for l in f]
        line_num = 0
        while line_num<len(lines):
            img_path = lines[line_num]

            img_dir = img_path.rpartition("/")[0]
            img_name = img_path.rpartition("/")[-1]
            os.makedirs(dir_path + output_dir + bbox_faces_dir + img_dir, exist_ok=True)
            img = cv2.imread(dir_path + images_dir + img_path + img_format)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype('float')
            img_width, img_height = img.shape
            
            num_faces = int(lines[line_num+1])
            for i in range(num_faces):
                left_x, left_y, right_x, right_y = [int(j) for j in lines[line_num+2+i].split()]
                crop_img = img[left_y:right_y, left_x:right_x]
                cv2.imwrite(dir_path + output_dir + bbox_faces_dir + \
                            img_dir + "/" + img_name + "_" + str(i) + img_format, \
                            cv2.resize(crop_img, face_dim))
            line_num += num_faces + 2

In [9]:
extract_faces_from_images()

In [10]:
def generate_non_face_image_coordinates(face_coordinates, img_width, img_height):
    w, h = img_width, img_height
    if img_width<w and img_height<h:
        return set()
    
    non_face_cordinates = set()
    
    for x in range(img_width-20):
        for y in range(img_height-20):
            lx, ly, rx, ry = x, y, x+w, y+h
            for llx, lly, rrx, rry in face_coordinates:
                if ((rx<=llx) or (ry<=lly) or (lx >= rrx) or (ly >= rry)):
                    non_face_cordinates.add((lx,ly,rx,ry))
                    if len(non_face_cordinates) == len(face_coordinates):
                        return non_face_cordinates
    return non_face_cordinates

In [11]:
#extract non face data in the form of bbox
def extract_non_faces_from_images():
    for bounding_box_file in bounding_box_files:
        with open(dir_path + output_dir + bbox_fdd_folds + bounding_box_file) as f:
            lines = [l.strip("\n") for l in f]
        line_num = 0
        while line_num<len(lines):
            img_path = lines[line_num]

            img_dir = img_path.rpartition("/")[0]
            img_name = img_path.rpartition("/")[-1]
            os.makedirs(dir_path + output_dir + bbox_non_faces_dir + img_dir, exist_ok=True)
            img = cv2.imread(dir_path + images_dir + img_path + img_format)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype('float')
            img_width, img_height = img.shape
            
            num_faces = int(lines[line_num+1])
            face_coordinates = set()
            for i in range(num_faces):
                left_x, left_y, right_x, right_y = [int(j) for j in lines[line_num+2+i].split()]
                face_coordinates.add((left_x, left_y, right_x, right_y))  
            non_face_coordinates = generate_non_face_image_coordinates(face_coordinates, img_width, img_height)
 
            for idx, j in enumerate(non_face_coordinates):
                lx, ly, rx, ry = j
                crop_img = img[ly:ry, lx:rx]
                cv2.imwrite(dir_path + output_dir + bbox_non_faces_dir + \
                            img_dir + "/" + img_name + "_non_face_" + str(idx) + img_format, \
                            cv2.resize(crop_img, face_dim))
            line_num += num_faces + 2


In [12]:
extract_non_faces_from_images()